# ETL Script for Chicago Crime Database

####Importing

In [152]:
from google.cloud import bigquery
from datetime import datetime
import logging as log
import os

Storing the logs at 2021-12-15 21:39:27.236299 in this location:/Users/melodyfeng/Desktop/MSCA/31012 DE20211215-213927LoggingText.log


####Environmental variables and authentication 

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/melodyfeng/Downloads/crimeproject-335121-2c2943a752b5.json"

####Set up logging file and level of logging

In [ ]:
logFile = '{}{}{}'.format(r'/Users/melodyfeng/Desktop/MSCA/31012 DE',
                          datetime.now().strftime("%Y%m%d-%H%M%S"), r'LoggingText.log')
print("Storing the logs at {} in this location:{}".format(datetime.now(), logFile))

log.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=log.INFO, filename=logFile)

start_time = datetime.now()
log.info("Starting connection at: {}".format(start_time))

## Table 1: tbl_location

In [155]:
start_time = datetime.now()
log.info("Creating table 1: tbl_location at {}".format(start_time))

# Construct a BigQuery client object.

client = bigquery.Client()

query_job = client.query(
    """
        CREATE OR REPLACE TABLE `crimeproject-335121.chicago_crime.tbl_location` 
            (block_id INT64 NOT NULL
             , block STRING NOT NULL 
             , ward INT64 NOT NULL 
             , community_area INT64 NOT NULL 
            )
    """
)
results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time

log.info("Table tbl_location is created. It took {} for the process to complete".format(time_diff))

####Null values in tbl_location 

In [156]:
query_job = client.query(
    """
        SELECT 
            SUM(CASE WHEN block IS NULL THEN 1 ELSE 0 END) block_null
            , SUM(CASE WHEN ward IS NULL THEN 1 ELSE 0 END) ward_null 
            , SUM(CASE WHEN community_area IS NULL THEN 1 ELSE 0 END) community_area_null 
        FROM `crimeproject-335121.chicago_crime.crime_raw` 
    """
)

results = query_job.result()

col_names = ['block', 'ward', 'community_area']

result_dict = {col_names[i]:row[i] for row in results for i in range(len(col_names))}
        
log.info("These many nulls rows that will be deleted from the final table: {}".format(result_dict))


####Inserting data into tbl_location and creating block_id

In [157]:
start_time = datetime.now()

query_job = client.query(
    """
    INSERT `crimeproject-335121.chicago_crime.tbl_location`
    SELECT 
        ROW_NUMBER() OVER (ORDER BY block) AS block_id
        , block
        , ward
        , community_area
    FROM `crimeproject-335121.chicago_crime.crime_raw`
    WHERE block IS NOT NULL AND ward IS NOT NULL AND community_area IS NOT NULL 
    GROUP BY 2,3,4
    """
)

results = query_job.result()
end_time = datetime.now()
time_diff = end_time - start_time
log.info("Data has been inserted into tbl_location. It took {} time to complete the task".format(time_diff))


## Table 2: tbl_crime

In [158]:
start_time = datetime.now()
log.info("Creating table 2: tbl_crime at {}".format(start_time))

query_job = client.query(
    """
        CREATE OR REPLACE TABLE `crimeproject-335121.chicago_crime.tbl_crime` 
            (crime_id INT64 NOT NULL
             , case_number STRING NOT NULL 
             , date DATE NOT NULL 
             , fbi_code STRING NOT NULL 
             , arrest BOOLEAN NOT NULL
             , domestic BOOLEAN NOT NULL 
             , iucr STRING NOT NULL
             , beat INT64 NOT NULL 
             , block_id INT64 NOT NULL
             , latitude FLOAT64 
             , longitude FLOAT64
            )
        PARTITION BY DATE_TRUNC(date, MONTH)  
    """
)

results = query_job.result()
end_time = datetime.now()
time_diff = end_time - start_time
log.info("Table tbl_crime is created. It took {} for the process to complete".format(time_diff))

####Null values in tbl_crime 

In [159]:
query_job = client.query(
    """
        SELECT 
            SUM(CASE WHEN crime_id IS NULL THEN 1 ELSE 0 END) id_null
            , SUM(CASE WHEN case_number IS NULL THEN 1 ELSE 0 END) case_number_null 
            , SUM(CASE WHEN date IS NULL THEN 1 ELSE 0 END) date_null
            , SUM(CASE WHEN fbi_code IS NULL THEN 1 ELSE 0 END) fbi_code_null
            , SUM(CASE WHEN arrest IS NULL THEN 1 ELSE 0 END) arrest_null
            , SUM(CASE WHEN domestic IS NULL THEN 1 ELSE 0 END) domestic_null 
            , SUM(CASE WHEN iucr IS NULL THEN 1 ELSE 0 END) iucr_null
            , SUM(CASE WHEN beat IS NULL THEN 1 ELSE 0 END) beat_null 
            , SUM(CASE WHEN block_id IS NULL THEN 1 ELSE 0 END) block_null
            , SUM(CASE WHEN latitude IS NULL THEN 1 ELSE 0 END) latitude_null 
            , SUM(CASE WHEN longitude IS NULL THEN 1 ELSE 0 END) longitude_null
        FROM `crimeproject-335121.chicago_crime.tbl_crime` 
    """
)

results = query_job.result()

col_names = ['id', 'case_number', 'date', 'fbi_code', 'arrest', 'domestic', 'iucr',
             'beat', 'block', 'latitude', 'longitude']
result_dict = {col_names[i]:row[i] for row in results for i in range(len(col_names))}
log.info("These many nulls rows that will be deleted from the final table: {}".format(result_dict))

####Inserting values in tbl_crime 

In [160]:
start_time = datetime.now()

query_job = client.query(
    """
    INSERT `crimeproject-335121.chicago_crime.tbl_crime`
    WITH cte_block AS (
        SELECT block_id, block 
        FROM `crimeproject-335121.chicago_crime.tbl_location`
    )
    SELECT 
        CAST(id AS INTEGER) AS crime_id 
        , case_number 
        , CAST(REPLACE(SUBSTR(TRIM(REPLACE(REPLACE(date, 'AM', ''), 'PM', '')), 1,10), '/', '-') as date format 'MM-DD-YYYY') AS date
        , fbi_code
        , CAST(arrest AS BOOLEAN) AS arrest 
        , CAST(domestic AS BOOLEAN) AS domestic
        , iucr
        , CAST(beat AS INTEGER) beat 
        , block_id
        , CAST(latitude AS NUMERIC) AS latitude
        , CAST(longitude AS NUMERIC) AS longitude
    FROM `crimeproject-335121.chicago_crime.crime_raw` cct
    JOIN cte_block ON cte_block.block = cct.block
    WHERE id IS NOT NULL AND case_number IS NOT NULL AND date IS NOT NULL AND fbi_code IS NOT NULL AND arrest IS NOT NULL 
        AND domestic IS NOT NULL AND iucr IS NOT NULL AND beat IS NOT NULL AND block_id IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11 
    """
)

results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time
log.info("Data has been inserted into tbl_crime. It took {} time to complete the task".format(time_diff))

## Table 3: tbl_iucr

In [161]:
start_time = datetime.now()
log.info("Creating table 3: tbl_iucr at {}".format(start_time))

query_job = client.query(
    """
        CREATE OR REPLACE TABLE `crimeproject-335121.chicago_crime.tbl_iucr` 
            (iucr STRING NOT NULL 
             , primary_type STRING NOT NULL
             , description STRING NOT NULL
            )  
    """
)
results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time

log.info("Table tbl_iucr is created. It took {} for the process to complete".format(time_diff))

####Null values in tbl_iucr

In [162]:
query_job = client.query(
    """
        SELECT 
            SUM(CASE WHEN iucr IS NULL THEN 1 ELSE 0 END) iucr_null
            , SUM(CASE WHEN primary_type IS NULL THEN 1 ELSE 0 END) primary_type_null 
            , SUM(CASE WHEN description IS NULL THEN 1 ELSE 0 END) description_null
        FROM `crimeproject-335121.chicago_crime.crime_raw` 
    """
)

results = query_job.result()

col_names = ['iucr', 'primary_type', 'description']

result_dict = {col_names[i]:row[i] for row in results for i in range(len(col_names))}

log.info("These many nulls rows that will be deleted from the final table: {}".format(result_dict))


####Inserting values into tbl_iucr 

In [141]:
start_time = datetime.now()

query_job = client.query(
    """
    INSERT `crimeproject-335121.chicago_crime.tbl_iucr`
    SELECT
            base.iucr
            , base.primary_type
            , base.description
        FROM (
        SELECT 
            iucr
            , primary_type
            , description
            , ROW_NUMBER() OVER (PARTITION BY iucr) AS row
        FROM `crimeproject-335121.chicago_crime.crime_raw`
        WHERE iucr IS NOT NULL AND primary_type IS NOT NULL AND description IS NOT NULL
        ) AS base
        WHERE base.row = 1
        GROUP BY 1,2,3
    """
)

results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time
log.info("Data has been inserted into tbl_iucr. It took {} time to complete the task".format(time_diff))

## Table 4: tbl_police_district

In [142]:
start_time = datetime.now()
log.info("Creating table 4: tbl_police_district at {}".format(start_time))

query_job = client.query(
    """
        CREATE OR REPLACE TABLE `crimeproject-335121.chicago_crime.tbl_police_district` 
            (beat INT64 NOT NULL 
             , district_id INT64 NOT NULL
            )  
    """
)
results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time

log.info("Table tbl_police_district is created. It took {} for the process to complete".format(time_diff))


####Null values in tbl_police_district

In [143]:
query_job = client.query(
    """
        SELECT 
            SUM(CASE WHEN beat IS NULL THEN 1 ELSE 0 END) beat_null
            , SUM(CASE WHEN district IS NULL THEN 1 ELSE 0 END) district_id_null 
        FROM `crimeproject-335121.chicago_crime.crime_raw` 
    """
)

results = query_job.result()

col_names = ['beat', 'district_id']

result_dict = {col_names[i]:row[i] for row in results for i in range(len(col_names))}

log.info("These many nulls rows that will be deleted from the final table: {}".format(result_dict))

####Inserting data into tbl_police_district

In [144]:
start_time = datetime.now()

query_job = client.query(
    """
    INSERT `crimeproject-335121.chicago_crime.tbl_police_district`
    SELECT 
        CAST(beat AS INTEGER) AS beat
        , CAST(district AS INTEGER) AS district_id 
    FROM `crimeproject-335121.chicago_crime.crime_raw`
    WHERE beat IS NOT NULL AND district IS NOT NULL
    GROUP BY 1,2
    """
)

results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time
log.info("Data has been inserted into tbl_police_district. It took {} time to complete the task".format(time_diff))

## Table 5 tbl_police_sentiment

In [145]:
start_time = datetime.now()
log.info("Creating table 5: tbl_police_sentiment at {}".format(start_time))

query_job = client.query(
    """
        CREATE OR REPLACE TABLE `crimeproject-335121.chicago_crime.tbl_police_sentiment` 
            (district_id INT64 NOT NULL 
             , overall_safety FLOAT64 NOT NULL
             , overall_trust FLOAT64 NOT NULL
             , safety_low_income FLOAT64 NOT NULL
             , safety_medium_income FLOAT64 NOT NULL
             , safety_high_income FLOAT64 NOT NULL
             , trust_low_income FLOAT64 NOT NULL
             , trust_medium_income FLOAT64 NOT NULL
             , trust_high_income FLOAT64 NOT NULL
            )  
    """
)
results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time

log.info("Table tbl_police_sentiment is created. It took {} for the process to complete".format(time_diff))

####Null values in tbl_police_sentiment

In [147]:
query_job = client.query(
    """
        SELECT 
            SUM(CASE WHEN district IS NULL THEN 1 ELSE 0 END) district_null
            , SUM(CASE WHEN safety IS NULL THEN 1 ELSE 0 END) safety_null
            , SUM(CASE WHEN trust IS NULL THEN 1 ELSE 0 END) trust_null
            , SUM(CASE WHEN s_income_low IS NULL THEN 1 ELSE 0 END) s_income_low_null
            , SUM(CASE WHEN s_income_medium IS NULL THEN 1 ELSE 0 END) s_income_medium_null
            , SUM(CASE WHEN s_income_high IS NULL THEN 1 ELSE 0 END) s_income_high_null
            , SUM(CASE WHEN t_income_low IS NULL THEN 1 ELSE 0 END) t_income_low_null
            , SUM(CASE WHEN t_income_medium IS NULL THEN 1 ELSE 0 END) t_income_medium_null
            , SUM(CASE WHEN t_income_high IS NULL THEN 1 ELSE 0 END) t_income_high_null
        FROM `crimeproject-335121.chicago_crime.police_sentiment_raw` 
    """
)

results = query_job.result()

result_dict = {}
col_names = ['district_id', 'overall_safety', 'overall_trust', 'safety_low_income', 'safety_medium_income',
             'safety_high_income', 'trust_low_income', 'trust_medium_income', 'trust_high_income']

result_dict = {col_names[i]:row[i] for row in results for i in range(len(col_names))}

log.info("These many nulls rows that will be deleted from the final table: {}".format(result_dict))

####Inserting data into tbl_police_sentiment

In [150]:
start_time = datetime.now()

query_job = client.query(
    """
    INSERT `crimeproject-335121.chicago_crime.tbl_police_sentiment`
    SELECT 
            district AS district_id 
            , AVG(safety) AS overall_safety
            , AVG(trust) AS overall_trust
            , AVG(s_income_low) AS safety_low_income
            , AVG(s_income_medium) AS safety_medium_income
            , AVG(s_income_high) AS safety_high_income
            , AVG(t_income_low) AS trust_low_income
            , AVG(t_income_medium) AS trust_medium_income
            , AVG(t_income_high) AS trust_high_income
        FROM `crimeproject-335121.chicago_crime.police_sentiment_raw`
        WHERE district IS NOT NULL AND safety IS NOT NULL AND trust IS NOT NULL AND s_income_low IS NOT NULL 
            AND s_income_medium IS NOT NULL AND s_income_high IS NOT NULL AND t_income_low IS NOT NULL AND t_income_medium IS NOT NULL 
            AND t_income_high IS NOT NULL 
        GROUP BY 1
    """
)

results = query_job.result()

end_time = datetime.now()
time_diff = end_time - start_time
log.info("Data has been inserted into tbl_police_sentiment. It took {} time to complete the task".format(time_diff))